In [54]:
import pandas as pd
import numpy as np

# fileName = open(r"D:/Python/毕设实验部分/data/临时特征集/特征集_209维特征_已做特征选择_零填充_已做过采样.csv")
# data = pd.read_csv(fileName)
# data.head()


feature = pd.read_table('/Users/tuyu/000000毕业设计/Flight_Delay/Train_Feature/final_feature.csv',sep=',',encoding='gb2312')

del(feature['本时段离港平均延误时长'])
del(feature['上一时段离港平均延误时长'])
del(feature['航班编号'])
# 1.前序延误 = 前序航班 实际到达时间 - 计划到达时间
# 2.起飞间隔 = 当前航班的计划起飞时间 - 前序航班实际到达时间
feature['起飞间隔'] = feature['起飞间隔'].fillna(0)
feature['前序延误'] = feature['前序延误'].fillna(0)
feature['出发机场天气得分'] = feature['出发机场天气得分'].interpolate()
feature['到达机场天气得分'] = feature['到达机场天气得分'].interpolate()
feature['平均延误时间'] = feature['平均延误时间'].interpolate()
feature['最大延误时间'] = feature['最大延误时间'].interpolate()
feature['延误时间中位数'] = feature['延误时间中位数'].interpolate()
feature['延误时间标准差'] = feature['延误时间标准差'].interpolate()

feature['本时段计划离港航班数'] = feature['本时段计划离港航班数'].fillna(feature['本时段计划离港航班数'].mean())
feature['本时段实际离港航班数'] = feature['本时段实际离港航班数'].fillna(feature['本时段实际离港航班数'].mean())
feature['本时段进港延误时长'] = feature['本时段进港延误时长'].fillna(feature['本时段进港延误时长'].mean())
feature['本时段实际进港延误航班数'] = feature['本时段实际进港延误航班数'].fillna(feature['本时段实际进港延误航班数'].mean())
feature['本时段离港延误时长'] = feature['本时段离港延误时长'].fillna(feature['本时段离港延误时长'].mean())
feature['本时段离港延误率'] = feature['本时段离港延误率'].fillna(feature['本时段离港延误率'].mean())
feature['上一时段进港延误时长'] = feature['上一时段进港延误时长'].fillna(feature['上一时段进港延误时长'].mean())
feature['上一时段离港延误时长'] = feature['上一时段离港延误时长'].fillna(feature['上一时段离港延误时长'].mean())
feature['上一时段实际进港航班数'] = feature['上一时段实际进港航班数'].fillna(feature['上一时段实际进港航班数'].mean())
feature['上一时段进港延误率'] = feature['上一时段进港延误率'].fillna(feature['上一时段进港延误率'].mean())

# feature = feature.sample(frac=0.1, replace=False, random_state=None,axis=0)
feature0 = feature[feature['标签'] == 0]
feature0new = feature0.sample(frac=0.15, replace=False, random_state=None,axis=0)
feature1 = feature[feature['标签'] == 1]
data = pd.concat([feature0new,feature1],axis=0, ignore_index=True)


In [55]:
# train = data[data['ORIGIN'] == 'train']
# test = data[data['ORIGIN'] == 'test']
from sklearn.model_selection import train_test_split
train, test = train_test_split(data, test_size=0.2, random_state=7)

train_labels = np.array(train['标签'].astype(np.int32)).reshape((-1,))
test_labels = np.array(test['标签'].astype(np.int32)).reshape((-1,))

train = train.drop(['标签'], axis = 1)
test = test.drop(['标签'], axis = 1)

features = np.array(train)
test_features = np.array(test)
labels = train_labels[:]

print('Train shape:', train.shape)
print('Test shape:', test.shape)
train.head()

Train shape: (1292410, 35)
Test shape: (323103, 35)


,计划起飞时间,计划到达时间,计划起飞时刻,计划到达时刻,计划飞行时间,起飞间隔,前序延误,平均延误时间,最大延误时间,延误时间中位数,...,本时段计划离港航班数,本时段实际离港航班数,本时段进港延误时长,本时段实际进港延误航班数,本时段离港延误时长,本时段离港延误率,上一时段进港延误时长,上一时段离港延误时长,上一时段实际进港航班数,上一时段进港延误率
1352958,17231,17232,23,1,2.000000,-1.216667,-0.366667,1.333947,20.083333,0.100000,...,35.000000,35.000000,-6.283333,0.000000,19.583333,0.000000,-5.783333e+00,20.383333,15.633042,0.020608
976806,17083,17083,1,2,1.750000,0.750000,-0.083333,0.922169,10.000000,0.200000,...,15.393169,15.438519,-3.516667,0.257175,1.391369,0.019545,-4.500000e-01,2.531482,15.633042,0.020608
406338,17150,17150,13,15,2.250000,0.866667,0.216667,1.718756,10.000000,0.775000,...,11.000000,11.000000,-0.550000,0.000000,4.233333,0.000000,-1.783333e+00,11.316667,31.000000,0.000000
368992,16830,16830,2,3,1.250000,0.633333,0.200000,1.468756,11.966667,0.233333,...,1.000000,15.438519,-0.400000,0.000000,-0.016667,0.019545,-3.469447e-18,2.531482,15.633042,0.020608
629642,16953,16953,13,15,2.166667,1.200000,-0.116667,1.632762,10.000000,0.733333,...,17.000000,9.000000,14.050000,5.000000,48.816667,0.470588,8.533333e+00,33.283333,23.000000,0.178571


In [56]:
import lightgbm as lgb
import csv
from hyperopt import STATUS_OK
from timeit import default_timer as timer

train_set = lgb.Dataset(features, label = labels)
N_folds = 3
def objective(params, n_folds = N_folds):
    global ITERATION
    ITERATION += 1
    
    subsample = params['boosting_type'].get('subsample', 1.0)
    params['boosting_type'] = params['boosting_type']['boosting_type']
    params['subsample'] = subsample
    
    for parameter_name in ['num_leaves', 'subsample_for_bin', 'min_child_samples']:
        params[parameter_name] = int(params[parameter_name])
    start = timer()
    cv_results = lgb.cv(params, train_set, num_boost_round=100, nfold=10, metrics='auc',
          early_stopping_rounds = 100, verbose_eval = False, seed = 42)
    run_time = timer() - start
    
    best_score = np.max(cv_results['auc-mean'])
    loss = 1 - best_score
    n_estimators = int(np.argmax(cv_results['auc-mean']) + 1)
    
    of_connection = open(out_file, 'a')
    writer = csv.writer(of_connection)
    writer.writerow([loss, params, ITERATION, n_estimators, run_time])
    
    return {'loss': loss, 'params': params, 'iteration': ITERATION,
           'estimators': n_estimators, 'train_time': run_time, 'status': STATUS_OK}

In [57]:
from hyperopt import hp
from hyperopt.pyll.stochastic import sample
space = {'class_weight': hp.choice('class_weight', [None, 'balanced']),
        'boosting_type': hp.choice('boosting_type', [{'boosting_type': 'gbdt', 'subsample': hp.uniform('gdbt_subsample', 0.5, 1)},
                                                    {'boosting_type': 'dart', 'subsample': hp.uniform('dart_subsample', 0.5, 1)},
                                                    {'boosting_type': 'goss', 'subsample': 1.0}]),
        'num_leaves': hp.quniform('num_leaves', 30, 150, 1),
        'learning_rate': hp.loguniform('learning_rate', np.log(0.01), np.log(0.2)),
        'subsample_for_bin': hp.quniform('subsample_for_bin', 20000, 300000, 20000),
         'min_child_samples': hp.quniform('min_child_samples', 20, 500, 5),
         'reg_alpha': hp.uniform('reg_alpha', 0.0, 1.0),
         'reg_lambda': hp.uniform('reg_lambda', 0.0, 1.0),
         'colsample_bytree': hp.uniform('colsample_by_tree', 0.6, 1.0)}

In [58]:
from hyperopt import fmin
from hyperopt import tpe
tpe_algorithm = tpe.suggest

from hyperopt import Trials
bayes_trials = Trials()
out_file = 'gbm_trials.csv'
of_connection = open(out_file, 'w')
writer = csv.writer(of_connection)

writer.writerow(['loss', 'params', 'iteration', 'estimators', 'train_time'])
of_connection.close()
# Global variable
global  ITERATION

ITERATION = 0

max_evals = 2
# Run optimization
best = fmin(fn = objective, space = space, algo = tpe.suggest, max_evals = max_evals, trials = bayes_trials, rstate = np.random.RandomState(42))

# Sort the trials with lowest loss (highest AUC) first
bayes_trials_results = sorted(bayes_trials.results, key = lambda x: x['loss'])
bayes_trials_results[0]

[LightGBM] [Warning]                                 
Auto-choosing row-wise multi-threading, the overhead of testing was 0.090198 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info]                                    
Total Bins 4374                                      
[LightGBM] [Info]                                    
Number of data points in the train set: 1163169, number of used features: 35
[LightGBM] [Info]                                    
Using GOSS                                           
[LightGBM] [Warning]                                 
Auto-choosing row-wise multi-threading, the overhead of testing was 0.095720 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info]                                    
Total Bins 4374                                      
[LightGBM] [Info]    

Total Bins 4374                                                                   
[LightGBM] [Info]                                                                 
Number of data points in the train set: 1163169, number of used features: 35      
[LightGBM] [Info]                                                                 
Using GOSS                                                                        
[LightGBM] [Warning]                                                              
Auto-choosing col-wise multi-threading, the overhead of testing was 0.179975 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info]                                                                 
Total Bins 4374                                                                   
[LightGBM] [Info]                                                                 
Number of data points in the train set: 1163169, number of used features: 35      
[LightGBM] [Info]         

{'loss': 0.04997650730623171,
 'params': {'boosting_type': 'goss',
  'class_weight': None,
  'colsample_bytree': 0.7816680870637424,
  'learning_rate': 0.03457647873038711,
  'min_child_samples': 340,
  'num_leaves': 68,
  'reg_alpha': 0.9328679988478337,
  'reg_lambda': 0.727725743177325,
  'subsample_for_bin': 140000,
  'subsample': 1.0},
 'iteration': 2,
 'estimators': 100,
 'train_time': 141.4368505370221,
 'status': 'ok'}

In [59]:
#保存结果
results = pd.read_csv('gbm_trials.csv')
results.sort_values('loss', ascending = True, inplace = True)
results.reset_index(inplace = True, drop = True)
print(results.shape)
results.head()

(2, 5)


,loss,params,iteration,estimators,train_time
0,0.049977,"{'boosting_type': 'goss', 'class_weight': None...",2,100,141.436851
1,0.051551,"{'boosting_type': 'goss', 'class_weight': None...",1,100,142.784389


In [60]:
import ast
ast.literal_eval(results.loc[0, 'params'])
# 出于安全考虑，对字符串进行类型转换的时候，最好使用ast.literal_eval()函数, 而不是直接用eval()

{'boosting_type': 'goss',
 'class_weight': None,
 'colsample_bytree': 0.7816680870637424,
 'learning_rate': 0.03457647873038711,
 'min_child_samples': 340,
 'num_leaves': 68,
 'reg_alpha': 0.9328679988478337,
 'reg_lambda': 0.727725743177325,
 'subsample_for_bin': 140000,
 'subsample': 1.0}

In [61]:
best_bayes_estimators = int(results.loc[0, 'estimators'])
best_bayes_params = ast.literal_eval(results.loc[0, 'params']).copy()
best_bayes_model = lgb.LGBMClassifier(n_estimators=best_bayes_estimators, n_jobs=-1,
                                     objective='binary', **best_bayes_params, random_state=42)
best_bayes_model.fit(features, labels)

LGBMClassifier(boosting_type='goss', colsample_bytree=0.7816680870637424,
               learning_rate=0.03457647873038711, min_child_samples=340,
               num_leaves=68, objective='binary', random_state=42,
               reg_alpha=0.9328679988478337, reg_lambda=0.727725743177325,
               subsample_for_bin=140000)

In [62]:
import pickle
#保存模型

f= open('/Users/tuyu/000000毕业设计/Flight_Delay/Model/Bayes_model/best_bayes_LightGBM.pickle','wb')
pickle.dump(best_bayes_model,f)
f.close()

In [63]:
preds = best_bayes_model.predict(test_features)
preds

array([0, 1, 1, ..., 0, 0, 1], dtype=int32)

In [64]:
from sklearn.metrics import classification_report 
test_labels = test_labels.tolist()
test_labels = np.array([i for i in test_labels])
print(classification_report(preds, test_labels))

              precision    recall  f1-score   support

           0       0.96      0.90      0.93    225796
           1       0.80      0.91      0.85     97307

    accuracy                           0.91    323103
   macro avg       0.88      0.91      0.89    323103
weighted avg       0.91      0.91      0.91    323103



In [65]:
from sklearn import metrics
testAccuracy = metrics.accuracy_score(test_labels, preds)
print("验证样本的准确率: %.4f" % testAccuracy)


def ks(y_predicted, y_true):
    label=y_true
    #label = y_true.get_label()
    fpr,tpr,thres = metrics.roc_curve(label,y_predicted,pos_label=1)
    return 'ks',abs(fpr - tpr).max()
ks_score = ks(preds, test_labels)
print(ks_score) 

验证样本的准确率: 0.9067
('ks', 0.7642466998059849)


In [66]:
prediction_test_predprob = best_bayes_model.predict_proba(test_features)[:,1]

lgb_offline=(prediction_test_predprob >= 0.35) *1
from sklearn.metrics import classification_report 
print(classification_report(lgb_offline, test_labels))

              precision    recall  f1-score   support

           0       0.93      0.92      0.93    214856
           1       0.85      0.86      0.86    108247

    accuracy                           0.90    323103
   macro avg       0.89      0.89      0.89    323103
weighted avg       0.90      0.90      0.90    323103



In [81]:
#重新构造测试数据
test['预测值'] = preds
test['标签'] = test_labels
dataError = test[test['标签'] != test['预测值']]
dataError

,计划起飞时间,计划到达时间,计划起飞时刻,计划到达时刻,计划飞行时间,起飞间隔,前序延误,平均延误时间,最大延误时间,延误时间中位数,...,本时段进港延误时长,本时段实际进港延误航班数,本时段离港延误时长,本时段离港延误率,上一时段进港延误时长,上一时段离港延误时长,上一时段实际进港航班数,上一时段进港延误率,预测值,标签
1465970,17215,17215,3,6,3.333333,1.616667,-0.366667,0.658844,7.400000,0.233333,...,10.550000,0.000000,2.083333,0.000000,2.866667,2.700000,29.000000,0.000000,0,1
1567359,16913,16913,1,3,1.416667,12.950000,-0.533333,0.830704,10.000000,0.050000,...,-2.133333,0.257175,1.391369,0.019545,-0.566667,2.531482,15.633042,0.020608,0,1
1603119,17169,17169,11,13,1.500000,-1.200000,2.116667,1.309294,10.000000,0.316667,...,5.416667,3.000000,10.883333,0.066667,1.883333,9.300000,19.000000,0.095238,0,1
1536619,16894,16894,11,13,1.666667,-1.566667,2.566667,0.802233,10.000000,0.300000,...,-0.400000,0.000000,3.516667,0.125000,2.016667,5.600000,9.000000,0.000000,0,1
1455481,17110,17110,7,9,2.083333,0.183333,0.150000,0.736416,12.883333,0.233333,...,0.366667,0.000000,8.383333,0.000000,7.083333,10.733333,4.000000,0.000000,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1428786,16848,16848,1,2,0.750000,-0.433333,0.266667,0.960935,10.000000,0.200000,...,-3.331113,0.257175,2.016667,0.000000,-5.342989,0.366667,15.633042,0.020608,0,1
1547718,17058,17058,11,16,4.833333,1.650000,-0.400000,1.068836,10.000000,0.250000,...,16.133333,0.000000,5.966667,0.040000,12.500000,7.200000,92.000000,0.000000,0,1
1354914,17002,17002,0,2,2.250000,-0.816667,-0.433333,0.567647,10.000000,0.266667,...,-0.200000,0.257175,3.650000,0.000000,0.066667,2.531482,15.633042,0.020608,0,1
778010,17013,17013,8,10,1.583333,-1.533333,2.450000,1.698457,10.000000,1.258333,...,4.933333,0.000000,0.416667,0.000000,6.800000,2.531482,11.000000,0.000000,1,0


In [88]:
dataError = dataError.sample(n=15000,replace=False, random_state=None,axis=0)
dataTrue = test[test['标签'] == test['预测值']]
dataTrue1= dataTrue[dataTrue['标签']==1]

dataTrue1 = dataTrue.sample(n=15140,replace=False, random_state=None,axis=0)
dataTrue =pd.concat([dataTrue, dataTrue1], axis=0, ignore_index=True)

In [89]:
test_data_new = pd.concat([dataTrue, dataError], axis=0, ignore_index=True)
test_labels_new = np.array(test_data_new['标签'].astype(np.int32)).reshape((-1,))
test_data_new = test_data_new.drop(['预测值','标签'],axis=1)
test_features_new = np.array(test_data_new)


In [90]:
preds_new = best_bayes_model.predict(test_features_new)
from sklearn.metrics import classification_report 
test_labels_new = test_labels_new.tolist()
test_labels_new = np.array([i for i in test_labels_new])
print(classification_report(preds_new, test_labels_new))

              precision    recall  f1-score   support

           0       0.98      0.95      0.97    225699
           1       0.90      0.96      0.93     97404

    accuracy                           0.95    323103
   macro avg       0.94      0.95      0.95    323103
weighted avg       0.96      0.95      0.95    323103



In [91]:
from sklearn import metrics
testAccuracy = metrics.accuracy_score(test_labels_new, preds_new)
print("验证样本的准确率: %.4f" % testAccuracy)
print(ks_score) 

验证样本的准确率: 0.9536
('ks', 0.7642466998059849)
